<h1> Multi Label toxic comments using RNN(LSTM)</h1>

In [1]:
# %pip install tensorflow pandas matplotlib scikit-learn pandas numpy

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df = pd.read_csv('data//train.csv')

In [3]:
df.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.shape

(159571, 8)

In [5]:
df['id'].nunique()

159571

In [6]:
df.query('identity_hate == 1')

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
105,00472b8e2d38d1ea,A pair of jew-hating weiner nazi schmucks.,1,0,1,0,1,1
176,006b94add72ed61c,I think that your a Fagget get a oife and burn...,1,0,1,1,1,1
218,008e0818dde894fb,"Kill all niggers. \n\nI have hard, that others...",1,0,1,0,1,1
238,0097dd5c29bf7a15,u r a tw@ fuck off u gay boy.U r smelly.Fuck u...,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...
159281,fb726deec64157bd,LoL!! \n\nyou're GAY!! you will never know how...,1,1,1,0,1,1
159336,fc3efa2f6f025f6d,"Oh, fuck off. The pansy Jew would just whine a...",1,0,1,0,1,1
159400,fd052883fa6a8697,"Shalom \n\nSemite, get the fuck out of here. I...",1,1,1,1,1,1
159449,fdce660ddcd6d7ca,I think he is a gay fag!!!,1,0,0,0,0,1


In [7]:
df.head(1)['comment_text']

0    Explanation\nWhy the edits made under my usern...
Name: comment_text, dtype: object

In [8]:
df.query('identity_hate == 1')

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
105,00472b8e2d38d1ea,A pair of jew-hating weiner nazi schmucks.,1,0,1,0,1,1
176,006b94add72ed61c,I think that your a Fagget get a oife and burn...,1,0,1,1,1,1
218,008e0818dde894fb,"Kill all niggers. \n\nI have hard, that others...",1,0,1,0,1,1
238,0097dd5c29bf7a15,u r a tw@ fuck off u gay boy.U r smelly.Fuck u...,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...
159281,fb726deec64157bd,LoL!! \n\nyou're GAY!! you will never know how...,1,1,1,0,1,1
159336,fc3efa2f6f025f6d,"Oh, fuck off. The pansy Jew would just whine a...",1,0,1,0,1,1
159400,fd052883fa6a8697,"Shalom \n\nSemite, get the fuck out of here. I...",1,1,1,1,1,1
159449,fdce660ddcd6d7ca,I think he is a gay fag!!!,1,0,0,0,0,1


In [9]:
df.columns[2:]                                             

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

### Preprocessing

In [25]:
from tensorflow.keras.layers import TextVectorization

In [26]:
x = df['comment_text'] # all the text data(the input values)
y = df[df.columns[2:]].values # labels for each text data as an array of integers(the target values)

In [27]:
MAX_FEATURES = 200000 # No. of unique words to be stored in the model.

In [28]:
vectorizer  = TextVectorization(max_tokens = MAX_FEATURES,
                                output_sequence_length = 1800,
                                output_mode = 'int')  
# max_tokens - Total number of tokens/words to store 
# output_sequance_length - Max length of one sentance(no. of words)
# output_mode = int - the words are mapped to integers.

In [29]:
# the textVectorization lowercases and removes punctuation.

In [30]:
vectorizer.adapt(x.values) # x.values converts df into numpy array
# .adapt is ued to build a vocabulary (does a little bit of preprocessing for us)
# the vocabulary is ordered by the frequency of that word.

In [31]:
vectorizer('Hello World ,  opop you')[:4] # '1' token means tht the word was not seen at adapt time.
# 0 means that there is no word at that position

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([288, 263,   1,   7], dtype=int64)>

In [32]:
# import joblib
# joblib.dump(vectorizer,"vectorizer.pkl")

In [33]:
vectorized_text = vectorizer(x.values)
vectorized_text #(159571 - no. of sentences,1800 - max no. of words in a sentence set by us)

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

Creating a Data pipeline

In [34]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y)) # create a tf.data.datset object where input is vectorized text and output is multilabels (supervised learning)
dataset = dataset.cache()  # catches the dataset in memory which saves time as we dont have to load data  from the disk after each epoch 
dataset = dataset.shuffle(160000)  # shuflle the dataset so that NN does not earn the order of input which may result in overfitting
dataset = dataset.batch(16) # create batches of the datset helps in parallel processing 16 i sthe batch size
dataset = dataset.prefetch(8) # tells how many batches have to be prepared before the end of execution of current batch

In [35]:
train = dataset.take(int(len(dataset)*0.7))  # get the first 70% of the data as train data take takes in the number of batches to extract
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2)) # skip the fisrt 70% as it is training data, of the 30% first 20% is taken as validation data
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1)) # the data after 90% is left for testing

In [36]:
train_generator = train.as_numpy_iterator()
batch_x,batch_y  = train_generator.next() # numpy generator is an iterator that can be used to iterate through the batches in iterator

In [37]:
batch_x.shape # 16 - Batch size ,1800 - make words per sentence cap set by us


(16, 1800)

In [38]:
batch_y.shape # 16 batch size ,6 number of labels            

(16, 6)

In [39]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

### Create Sequential Model

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional,Dense,Embedding

In [23]:
model = Sequential() # instantiate a Sequntial Model

# embedding layer
model.add(Embedding(MAX_FEATURES+1,32)) # +1 to handle out-of-vocab tokens that might appear later in real world scenarios 
# 32 - no. of features the embedding of one token contains


#Bidirectional LSTM 
model.add(Bidirectional(LSTM(32,activation = 'tanh')))

# three dense(fully connected) layers that will do feature extraction
model.add(Dense(128,activation = 'relu'))
model.add(Dense(256,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))

# final layer 
model.add(Dense(6,activation = 'sigmoid')) # why 6 nodes - becasue the final output has 6 classes and because we are classifying the activation func is sigmoid
# sigmoid converts any value between 0 and 1 which actually is our output format for the 6 labels

In [24]:
model.compile(loss = 'BinaryCrossentropy',optimizer = 'Adam')

In [25]:
model.summary() 
# Why LSTM has 64 nodes because two LSTM have been created due to it being bidirectional (one for forward propogation and one for back)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


Only comment if executing first time.

In [27]:
# history = model.fit(train,epochs = 10,validation_data = val )

In [28]:
# model.save('biltsm.h5')

### Loading the Model

In [40]:
from tensorflow.keras.models import load_model
model = load_model('biltsm.h5') 

In [41]:
from matplotlib import pyplot as plt

In [42]:
# plt.figure(figsize = (10,10))
# pd.DataFrame(history.history).plot()
# plt.show()

### Evaluate The Model

In [43]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [44]:
pre = Precision()
rec = Recall()
acc = CategoricalAccuracy()

In [45]:
for batch in test.as_numpy_iterator():  #the test dataset has some no. of batches of size 16 each
    x_test,y_test = batch
    y_pred = (model.predict(x_test)>0.5).astype(int)
    
    y_test = y_test.flatten()
    y_pred = y_pred.flatten()
    
    pre.update_state(y_test,y_pred) # after calculating for the particular batch it updates the overall metrics for all the batches passed till that point 
    rec.update_state(y_test,y_pred) # at the end we will have these metric values for the entire test database even though
    acc.update_state(y_test,y_pred)
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 862ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [46]:
print(f"Precision = {pre.result().numpy()}, Recall = {rec.result().numpy()},Accuracy = {acc.result().numpy()}")

Precision = 0.9324572086334229, Recall = 0.9053412675857544,Accuracy = 0.9227682948112488


### Make Predictions 

In [47]:
input = "Fuck you Nigga"

In [48]:
input_embed = vectorizer(input)

In [49]:
input_embed

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([ 139,    7, 3820, ...,    0,    0,    0], dtype=int64)>

In [50]:
predictions = model.predict(np.expand_dims(input_embed,0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 730ms/step


In [51]:
predictions[0]

array([0.9997611 , 0.33097976, 0.99856836, 0.00956304, 0.98726356,
       0.913736  ], dtype=float32)

In [52]:
labels  = df.columns[2:]

In [53]:
labels

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [54]:
for i in range(0,len(labels)):
    print(f"{labels[i]}:{predictions[0][i]>0.5}")

toxic:True
severe_toxic:False
obscene:True
threat:False
insult:True
identity_hate:True


In [55]:
prediction = dict(zip(labels,predictions))
prediction

{'toxic': array([0.9997611 , 0.33097976, 0.99856836, 0.00956304, 0.98726356,
        0.913736  ], dtype=float32)}

In [56]:
# Weights for each label
weights = {
    "toxic": 10,
    "severe_toxic": 10,
    "obscene": 10,
    "threat": 10,
    "insult": 10,
    "identity_hate":10
}
prediction = dict(zip(labels,predictions[0]))

# Step 1: Compute Negativity Score
negativity_score = sum(prediction[label] * weights[label] for label in labels)

# Step 2: Map to a range of 1 to 10
max_possible_score = sum(weights.values())  # When all probabilities are 1
final_score = 10 - (negativity_score / max_possible_score)*9

# Step 3: Clip the score to [1, 10]
final_score = np.clip(final_score, 1, 10)

print(f"Final Score: {final_score:.2f}")


Final Score: 3.64


In [57]:
from pydantic import BaseModel

class comment(BaseModel):
    comment : str


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
nest_asyncio.apply()

app = FastAPI()
origins = [
    "http://localhost:3000",  # Next.js frontend running on port 3000
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get('/')
def index():
    return{'message':'hello,stranger'}
@app.post('/predict')
def get_score(data:comment):
    data = data.dict()
    comment = data['comment']
    vectorized_text = vectorizer(comment)
    predictions = model.predict(np.expand_dims(vectorized_text,0))
    prediction = dict(zip(labels,predictions[0]))

    # Weights for each label
    weights = {
        "toxic": 7,
        "severe_toxic": 10,
        "obscene": 8,
        "threat": 9,
        "insult": 9,
        "identity_hate":10
    }

    # Step 1: Compute Negativity Score
    negativity_score = sum(prediction[label] * weights[label] for label in labels)

    # Step 2: Map to a range of 1 to 10
    max_possible_score = sum(weights.values())  # When all probabilities are 1
    final_score = 10 - (negativity_score / max_possible_score)*10

    # Step 3: Clip the score to [1, 10]
    final_score = np.clip(final_score, 1, 10)
    # print(f"Final score is {final_score}")
    return {
        'score':f'{final_score}'
    }


if __name__ == "__main__":
    uvicorn.run(app,host = '127.0.0.1',port  = 5000)

INFO:     Started server process [33544]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:49470 - "OPTIONS /predict HTTP/1.1" 200 OK


C:\Users\Pratham\AppData\Local\Temp\ipykernel_33544\2795886810.py:25: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  data = data.dict()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
INFO:     127.0.0.1:49470 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━